In [97]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

def RMSEvec(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        if ground[i] != 0:
            error += (ground[i] - predict[i])**2
            n += 1
                
    return np.sqrt(error/n)

def meanError(ground_truth,new_res):
    return np.mean(abs((new_res - ground_truth)[ground_truth!=0]))

In [99]:
def dictfromR(R):

    R_dict = {"Users": np.empty([0]), "Movies": np.empty([0]), "Ratings": np.empty([0])}

    for i in range(len(R)):
        for j in range(len(R[0])):
            if R[i,j] != 0:
                R_dict["Users"] = np.append(R_dict["Users"],i)
                R_dict["Movies"] = np.append(R_dict["Movies"],j)
                R_dict["Ratings"] = np.append(R_dict["Ratings"],R[i,j])

    return R_dict

In [100]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/DATA/MVA/Graphes/Projet/cf-cold-start/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.])}


In [101]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [102]:
np.argmax([np.sum(P[i,:] for i in range(len(P)))][0])

321

In [103]:
to_keep = 3
sel = np.where(R[:,321] != 0)[0]
np.random.shuffle(sel)
sel = sel[:len(sel)-to_keep]

In [104]:
ground_truth = copy.deepcopy(R[:,321])
R[sel,321] = 0

In [105]:
np.sum(R[:,321] != 0)

3

In [106]:
R_dictCopy = copy.deepcopy(R_dict)
R_dict = dictfromR(R)

In [107]:
np.where(R_dict['Movies'] == 321)

(array([32894, 41209, 86143]),)

In [108]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [109]:
R_rec = np.dot(als.U,np.transpose(als.V))

In [110]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.69895785945
0.946232339213


In [111]:
print(RMSEvec(ground_truth, R_rec[:,321]))

0.875066692088


In [112]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [113]:
np.max(R_rec)

8.8455390430241945

In [114]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [115]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [116]:
supp = 100
test_vec = copy.deepcopy(R[:,321])*2
# test_vec[:supp] = [0 for i in range(supp)]
test_vec.shape

(671,)

In [117]:
# test_vec

In [118]:
hfs0, confidence = simple_hfs(als.U, test_vec, L, sim)
# hfs0/2

/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [119]:
maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])

In [120]:
lim = np.percentile(maxconfidences, 1)
print(RMSEvec(ground_truth*(maxconfidences > lim),hfs0/2))
print(RMSEvec(ground_truth*(maxconfidences > lim), R_rec[:,321]))

1.12657624744
0.874090978999


In [121]:
print(meanError(ground_truth,hfs0/2))
print(meanError(ground_truth,R_rec[:,321]))

0.802052785924
0.664169962001


In [122]:
# elmnt = 321
# val = []
# print(RMSEvec(R[:,elmnt],R_rec[:,elmnt]))
# for supp in range(1,671,10):
#     test_vec = copy.deepcopy(R[:,elmnt])*2
#     test_vec[:supp] = [0 for i in range(supp)]

#     hfs0 = simple_hfs(als.U, test_vec, L, sim)
#     val.append(RMSEvec(R[:,elmnt],hfs0/2))
    
# plt.plot(range(1,671,10),val)

In [136]:
lhfs = []
lconf = []
for i in range(len(R[0])):
    if i%1000 == 0:
        print(i)
    hfs0, confidence = simple_hfs(als.U, R[:,i]*2, L, sim)
    maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])
    
    lim = np.percentile(maxconfidences, 95)
    
    lhfs.append(hfs0/2)
    lconf.append(maxconfidences > lim)

R_barre = np.vstack(lhfs).T
confs = np.vstack(lconf).T
    
# R_barre[R_barre < 1] = .5
# R_barre[R_barre > 5] = 5

0


/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


1000
2000
3000
4000
5000
6000
7000
8000
9000


In [137]:
R_barre_limited = R_barre * confs

In [138]:
np.unique(R_barre_limited)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ])

In [139]:
print(RMSE(R_barre_limited,R_rec))

0.507726707474


In [140]:
R_barre_final = copy.deepcopy(R_barre_limited)
R_barre_final[R != 0] = 0

In [141]:
# R_dict_barre = dictfromR(R_barre_final)

In [145]:
N = len(R)
M = len(R[0])
K = 4

als_trans = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")

ans = als_trans.fitTransductive(R_dict,R_barre_final,C1=1,C2=0.2)

Als created


In [146]:
R_rec_trans = np.dot(als_trans.U,np.transpose(als_trans.V))
print(RMSE(R_rec_trans,R_rec))

0.3773409418


In [147]:
print(RMSEvec(ground_truth,R_rec_trans[:,321]))
print(RMSEvec(ground_truth,R_rec[:,321]))
print(RMSEvec(ground_truth,R_barre[:,321]))

0.899006597327
0.875066692088
1.12457225495


0.930864602802
0.97011494657
1.00036650175 (90 1 0.1)

0.910201784408
0.97011494657
1.00036650175 (90 1 0.5)

0.931048133417
0.97011494657
1.00036650175 (95 1 0.5)

In [135]:
R_barre

array([[ 3. ,  3. ,  3. , ...,  5. ,  3. ,  5. ],
       [ 5. ,  3. ,  4. , ...,  5. ,  3. ,  5. ],
       [ 3.5,  3. ,  4. , ...,  5. ,  3. ,  5. ],
       ..., 
       [ 4. ,  3. ,  3. , ...,  5. ,  3. ,  5. ],
       [ 4. ,  3. ,  5. , ...,  5. ,  3. ,  5. ],
       [ 5. ,  3.5,  3. , ...,  5. ,  3. ,  5. ]])